# Data Preparation


Take flights.csv and drop everything but : 

Our Xs (the only columns in flights_test.csv)
- **fl_date**
- **mkt_unique_carrier**
- **branded_code_share**
- **mkt_carrier**
- **mkt_carrier_fl_num**
- **op_unique_carrier**
- **tail_num**
- **op_carrier_fl_num**
- **origin_airport_id**
- **origin**
- **origin_city_name**
- **dest_airport_id**
- **dest**
- **dest_city_name**
- **crs_dep_time**
- **crs_arr_time**
- **dup**
- **crs_elapsed_time**
- **flights**
- **distance**

Our Ys : 

Q.1 = Regression
- **arr_delay**

Q.2 = Multiclass Classification
- **carrier_delay**
- **weather_delay**
- **nas_delay**
- **security_delay**
- **late_aircraft_delay**

Q.3 = Binary Classification
- **cancelled**


### Import Modules

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import datetime
import numpy as np
from sklearn import preprocessing 

### Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%%time
#Import data
#path = '../../../data/'
path = '/content/drive/My Drive/flights_data/'
#path = '/content/drive/My Drive/PERSO/LIGHTHOUSE LABS/MIDTERM/data/'
df = pd.read_csv(str(path)+"flights_sample_10.csv",low_memory=False)
df_test = pd.read_csv(str(path)+"flights_test.csv",low_memory=False)

CPU times: user 10.2 s, sys: 2.47 s, total: 12.7 s
Wall time: 19 s


In [ ]:
df.columns

Index(['Unnamed: 0', 'fl_date', 'mkt_unique_carrier', 'branded_code_share',
       'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [ ]:
print(df_test.columns)

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance'],
      dtype='object')


#### Keep only the tests columns and 7 predictors columns (+ dep_delay to infer historical data)

In [ ]:
dfF = df[['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time','dep_delay',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance','arr_delay','carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay','cancelled']].copy()
dfFT = df_test.copy()

#### Look for Nans

In [ ]:
total = dfF.isnull().sum().sort_values(ascending=False)
percent = (dfF.isnull().sum()/dfF.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data[missing_data['Total']>0]

,Total,Percent
security_delay,1291631,0.811100
nas_delay,1291631,0.811100
weather_delay,1291631,0.811100
carrier_delay,1291631,0.811100
late_aircraft_delay,1291631,0.811100
arr_delay,31241,0.019618
dep_delay,26581,0.016692
tail_num,5008,0.003145
crs_elapsed_time,6,0.000004


In [ ]:
total = dfFT.isnull().sum().sort_values(ascending=False)
percent = (dfFT.isnull().sum()/dfFT.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data[missing_data['Total']>0]

,Total,Percent
tail_num,1499,0.002269


#### Drop some Nans

In [ ]:
dfF = dfF[dfF.dep_delay.notnull()]
dfF = dfF[dfF.arr_delay.notnull()]

In [ ]:
#dfFT

## Feature engineering

#### Convert delay types to Boolean values

In [ ]:
#list of each delay type
delaytypes = ['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']

#creates new bool column for each delay type, where 1 if that delay type is the max of all delays AND is > 0
for delay in delaytypes:
    dfF['is'+delay] = np.where((dfF[delaytypes].idxmax(axis=1) == delay)&(dfF[delay]>0), 1, 0)


In [ ]:
#dfF

####  Create flight number

In [ ]:
# Not being used for now

In [ ]:
dfF['flight_number'] = dfF['mkt_unique_carrier']+dfF['mkt_carrier_fl_num'].astype(str)+dfF['origin']+dfF['dest']
dfFT['flight_number'] = dfFT['mkt_unique_carrier']+dfFT['mkt_carrier_fl_num'].astype(str)+dfFT['origin']+dfFT['dest']

In [ ]:
#dfF['flight_number'].value_counts()

In [ ]:
#dfFT['flight_number'].value_counts()

In [ ]:
#print('Number of different flight_numbers in train set : ' +str(len(dfF['flight_number'].value_counts())))
#print('Number of different flight_numbers in test set : ' +str(len(dfFT['flight_number'].value_counts())))

####  Convert time data to Month / Day / Day of year / Day of Week / Dep Hour / Arr Hour columns

In [ ]:
dfF.fl_date = pd.to_datetime(dfF.fl_date, format='%Y-%m-%d')
dfFT.fl_date = pd.to_datetime(dfFT.fl_date, format='%Y-%m-%d')

dfF['fl_month'] = pd.DatetimeIndex(dfF.fl_date).month
dfFT['fl_month'] = pd.DatetimeIndex(dfFT.fl_date).month

dfF['fl_day'] = pd.DatetimeIndex(dfF.fl_date).day
dfFT['fl_day'] = pd.DatetimeIndex(dfFT.fl_date).day

dfF['fl_day_of_year'] = dfF.fl_date.dt.dayofyear
dfFT['fl_day_of_year'] = dfFT.fl_date.dt.dayofyear

dfF['fl_day_of_week'] = dfF.fl_date.dt.dayofweek
dfFT['fl_day_of_week'] = dfFT.fl_date.dt.dayofweek

dfF['crs_dep_time'] = dfF['crs_dep_time'].apply(lambda x: '{0:0>4}'.format(x))
dfF['crs_arr_time'] = dfF['crs_arr_time'].apply(lambda x: '{0:0>4}'.format(x))
dfFT['crs_dep_time'] = dfFT['crs_dep_time'].apply(lambda x: '{0:0>4}'.format(x))
dfFT['crs_arr_time'] = dfFT['crs_arr_time'].apply(lambda x: '{0:0>4}'.format(x))

dfF['dep_hour'] = dfF['crs_dep_time'].str[:2]
dfF['arr_hour'] = dfF['crs_arr_time'].str[:2]

dfFT['dep_hour'] = dfFT['crs_dep_time'].str[:2]
dfFT['arr_hour'] = dfFT['crs_arr_time'].str[:2]

#### Create short, med, long haul bins

In [ ]:
# duration type (eg short, med, long haul)
bins = [-100, 180, 360, 2000]
labels = [1,2,3]
dfF['duration_type'] = pd.cut(dfF['crs_elapsed_time'], bins=bins, labels=labels)
dfFT['duration_type'] = pd.cut(dfFT['crs_elapsed_time'], bins=bins, labels=labels)

In [ ]:
#dfF

#### Creatd day sections bins

In [ ]:
hour_to_daysection = {'00':0,'01':0,'02':0,'03':0,'04':0,'05':0,'06':1,'07':1,'08':1,'09':1,'10':1,'11':2,'12':2,'13':2,'14':2,'15':2,'16':2,'17':3,'18':3,'19':3,'20':4,'21':4,'22':4,'23':4,'24': 4}

# Night = 0, Morning rush = 1, Midday = 2, Afternoon rush = 3, Evening = 4

dfF['dep_daysection'] = dfF['dep_hour'].map(hour_to_daysection)
dfFT['dep_daysection'] = dfFT['dep_hour'].map(hour_to_daysection)

dfF['arr_daysection'] = dfF['arr_hour'].map(hour_to_daysection)
dfFT['arr_daysection'] = dfFT['arr_hour'].map(hour_to_daysection)

#### Create Seasons bins

In [ ]:
month_to_season = {1: 1, 2: 1, 3: 2, 4: 2, 5: 2, 6: 3, 7: 3, 8: 3, 9: 4, 10: 4, 11: 4, 12: 1}
dfF['fl_season'] = dfF['fl_month'].map(month_to_season)
dfFT['fl_season'] = dfFT['fl_month'].map(month_to_season)

In [ ]:
#dfF

#### Create Origin and Destination State columns

In [ ]:
dfF['origin_state'] = dfF['origin_city_name'].str[-2:]
dfF['dest_state'] = dfF['dest_city_name'].str[-2:]

dfFT['origin_state'] = dfFT['origin_city_name'].str[-2:]
dfFT['dest_state'] = dfFT['dest_city_name'].str[-2:]

In [ ]:
#dfF

# Aggregate statistics values

#### Carrier Volume

In [ ]:
# total volume at origin and dest airport
dfF['origin_airport_vol'] = dfF.groupby(['origin_airport_id'])['flights'].transform(np.sum)
dfF['dest_airport_vol'] = dfF.groupby(['origin_airport_id'])['flights'].transform(np.sum)

# how much the airline operates out of that airport
dfF['carrier_origin_apt_vol'] = dfF.groupby(['origin_airport_id','op_unique_carrier'])['flights'].transform(np.sum)
dfF['carrier_dest_apt_vol'] = dfF.groupby(['dest_airport_id','op_unique_carrier'])['flights'].transform(np.sum)

# how much the airline operates out of that airport as pct
dfF['carrier_origin_apt_pct'] = dfF['carrier_origin_apt_vol'] / dfF['origin_airport_vol'] * 100
dfF['carrier_dest_apt_pct'] = dfF['carrier_dest_apt_vol'] / dfF['dest_airport_vol'] * 100

In [ ]:
# total volume at origin and dest airport
dfFT['origin_airport_vol'] = dfFT.groupby(['origin_airport_id'])['flights'].transform(np.sum)
dfFT['dest_airport_vol'] = dfFT.groupby(['origin_airport_id'])['flights'].transform(np.sum)

# how much the airline operates out of that airport
dfFT['carrier_origin_apt_vol'] = dfFT.groupby(['origin_airport_id','op_unique_carrier'])['flights'].transform(np.sum)
dfFT['carrier_dest_apt_vol'] = dfFT.groupby(['dest_airport_id','op_unique_carrier'])['flights'].transform(np.sum)

# how much the airline operates out of that airport as pct
dfFT['carrier_origin_apt_pct'] = dfFT['carrier_origin_apt_vol'] / dfFT['origin_airport_vol'] * 100
dfFT['carrier_dest_apt_pct'] = dfFT['carrier_dest_apt_vol'] / dfFT['dest_airport_vol'] * 100

#### Mean Carrier Delay

In [ ]:
dfF['mean_carrier_arr_delay'] = dfF.groupby(['op_unique_carrier'])['arr_delay'].transform(np.mean)
dfFT['mean_carrier_arr_delay'] = dfF.groupby(['op_unique_carrier'])['arr_delay'].transform(np.mean)

#### Carrier Delay per month

In [ ]:
stattype = [('mean', np.mean), ('median', np.median), ('std',np.std)]

for stat in stattype:
  # arr per airline per month
  dfF[stat[0]+'_carrier_arr_delay_month'] = dfF.groupby(['op_unique_carrier','fl_month'])['arr_delay'].transform(stat[1])
  dfFT[stat[0]+'_carrier_arr_delay_month'] = dfF.groupby(['op_unique_carrier','fl_month'])['arr_delay'].transform(stat[1])

  # dep per airline per month
  dfF[stat[0]+'_carrier_dep_delay_month'] = dfF.groupby(['op_unique_carrier','fl_month'])['dep_delay'].transform(stat[1])
  dfFT[stat[0]+'_carrier_dep_delay_month'] = dfF.groupby(['op_unique_carrier','fl_month'])['dep_delay'].transform(stat[1])

#### Carrier Delay per origin and hour

In [ ]:
stattype = [('mean', np.mean), ('median', np.median), ('std',np.std)]

for stat in stattype:

  ## Carrier-Airport-(DEP and ARR) delay per (DEP and ARR) hour
  dfF[stat[0]+'_carrier_origin_dep-hour_dep_delay'] = dfF.groupby(['op_unique_carrier','origin','dep_hour'])['dep_delay'].transform(stat[1])
  dfFT[stat[0]+'_carrier_origin_dep-hour_dep_delay'] = dfF.groupby(['op_unique_carrier','origin','dep_hour'])['dep_delay'].transform(stat[1])
  dfF[stat[0]+'_carrier_dest_fl-hour_delay'] = dfF.groupby(['op_unique_carrier','dest','arr_hour'])['arr_delay'].transform(stat[1])
  dfFT[stat[0]+'_carrier_dest_fl-hour_delay'] = dfF.groupby(['op_unique_carrier','dest','arr_hour'])['arr_delay'].transform(stat[1])


#### Carrier Delay per route and month/week day/daysection

In [ ]:
stattype = [('mean', np.mean), ('median', np.median), ('std',np.std)]

for stat in stattype:

  ## Mean DEP and ARR delay for Carrier routes per month per day of the week per day section. 

  dfF[stat[0]+'_carrier_route_month_dow_dayS_dep-delay'] = dfF.groupby(['op_unique_carrier','origin','dest','fl_month','fl_day_of_week','dep_daysection'])['dep_delay'].transform(stat[1])
  dfFT[stat[0]+'_carrier_route_month_dow_dayS_dep-delay'] = dfF.groupby(['op_unique_carrier','origin','dest','fl_month','fl_day_of_week','dep_daysection'])['dep_delay'].transform(stat[1])
  dfF[stat[0]+'_carrier_route_month_dow_dayS_arr-delay'] = dfF.groupby(['op_unique_carrier','origin','dest','fl_month','fl_day_of_week','arr_daysection'])['arr_delay'].transform(stat[1])
  dfFT[stat[0]+'_carrier_route_month_dow_dayS_arr-delay'] = dfF.groupby(['op_unique_carrier','origin','dest','fl_month','fl_day_of_week','arr_daysection'])['arr_delay'].transform(stat[1])

#### More mean/medians

In [ ]:
stattype = [('mean', np.mean), ('median', np.median), ('std',np.std)]

# arrival and departure delay in a list
delays = ['arr_delay','dep_delay']

#list of each slice of time as a tuple. 1st element is name for the new feature, 2nd is its existing name
times = [('month','fl_month'),('dow','fl_day_of_week'),('dep_hour','dep_daysection'),('arr_hour','arr_daysection')]

#loop-ception
for time in times:
  for stat in stattype:
    for delay in delays:
      dfF[stat[0]+'_route_'+delay+'_'+time[0]] = dfF.groupby(['origin','dest', time[1]])[delay].transform(stat[1])
      dfFT[stat[0]+'_route_'+delay+'_'+time[0]] = dfF.groupby(['origin','dest',time[1]])[delay].transform(stat[1])

In [ ]:
dfF

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_delay,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,cancelled,iscarrier_delay,isweather_delay,isnas_delay,issecurity_delay,islate_aircraft_delay,flight_number,fl_month,fl_day,fl_day_of_year,fl_day_of_week,dep_hour,arr_hour,duration_type,dep_daysection,arr_daysection,fl_season,origin_state,dest_state,origin_airport_vol,dest_airport_vol,carrier_origin_apt_vol,carrier_dest_apt_vol,carrier_origin_apt_pct,carrier_dest_apt_pct,mean_carrier_arr_delay,mean_carrier_arr_delay_month,mean_carrier_dep_delay_month,median_carrier_arr_delay_month,median_carrier_dep_delay_month,std_carrier_arr_delay_month,std_carrier_dep_delay_month,mean_carrier_origin_dep-hour_dep_delay,mean_carrier_dest_fl-hour_delay,median_carrier_origin_dep-hour_dep_delay,median_carrier_dest_fl-hour_delay,std_carrier_origin_dep-hour_dep_delay,std_carrier_dest_fl-hour_delay,mean_carrier_route_month_dow_dayS_dep-delay,mean_carrier_route_month_dow_dayS_arr-delay,median_carrier_route_month_dow_dayS_dep-delay,median_carrier_route_month_dow_dayS_arr-delay,std_carrier_route_month_dow_dayS_dep-delay,std_carrier_route_month_dow_dayS_arr-delay,mean_route_arr_delay_month,mean_route_dep_delay_month,median_route_arr_delay_month,median_route_dep_delay_month,std_route_arr_delay_month,std_route_dep_delay_month,mean_route_arr_delay_dow,mean_route_dep_delay_dow,median_route_arr_delay_dow,median_route_dep_delay_dow,std_route_arr_delay_dow,std_route_dep_delay_dow,mean_route_arr_delay_dep_hour,mean_route_dep_delay_dep_hour,median_route_arr_delay_dep_hour,median_route_dep_delay_dep_hour,std_route_arr_delay_dep_hour,std_route_dep_delay_dep_hour,mean_route_arr_delay_arr_hour,mean_route_dep_delay_arr_hour,median_route_arr_delay_arr_hour,median_route_dep_delay_arr_hour,std_route_arr_delay_arr_hour,std_route_dep_delay_arr_hour
0,2018-05-29,DL,DL,DL,551,DL,N945DL,551,10397,ATL,"Atlanta, GA",11995,GSO,"Greensboro/High Point, NC",2331,31.0,0039,N,68.0,1,306,25.0,0.0,0.0,0.0,0.0,25.0,0,0,0,0,0,1,DL551ATLGSO,5,29,149,1,23,00,1,4,0,2,GA,NC,77632,77632,48610,520,62.615932,0.669827,0.380758,0.475128,6.894228,-8.0,-2.0,39.112943,36.989578,16.668478,7.911765,-1.0,1.5,46.556648,24.201637,17.000000,13.000000,17.0,13.0,19.798990,16.970563,0.444444,2.533333,-4.0,-2.0,16.132344,11.173426,-4.957746,1.028169,-10.0,-3.0,17.946298,14.206610,7.257353,12.955882,-5.0,-1.0,42.798580,42.029067,7.911765,12.882353,1.5,3.0,24.201637,23.105509
1,2018-05-29,DL,DL,DL,565,DL,N918DH,565,14100,PHL,"Philadelphia, PA",11433,DTW,"Detroit, MI",0600,-6.0,0737,N,97.0,1,453,-14.0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,DL565PHLDTW,5,29,149,1,06,07,1,1,1,2,PA,MI,30118,30118,1285,11404,4.266552,37.864400,0.380758,0.475128,6.894228,-8.0,-2.0,39.112943,36.989578,3.293103,-7.638963,-3.0,-11.0,47.288443,34.517556,-9.666667,-15.666667,-9.0,-14.0,4.041452,2.886751,-0.625000,3.055556,-6.5,-4.0,29.014536,27.542484,-2.330275,3.192661,-9.0,-5.0,28.973824,26.710200,-5.790875,1.604563,-11.0,-4.0,30.512511,28.912138,-8.484472,-2.167702,-11.0,-5.0,14.455581,10.144109
2,2018-05-29,DL,DL,DL,658,DL,N990DL,658,10397,ATL,"Atlanta, GA",12448,JAN,"Jackson/Vicksburg, MS",1950,26.0,2010,N,80.0,1,341,43.0,18.0,0.0,17.0,0.0,8.0,0,1,0,0,0,0,DL658ATLJAN,5,29,149,1,19,20,1,3,4,2,GA,MS,77632,77632,48610,456,62.615932,0.587387,0.380758,0.475128,6.894228,-8.0,-2.0,39.112943,36.989578,13.427613,9.351351,0.0,-3.0,49.677859,33.409260,8.000000,8.666667,-1.0,-5.0,15.588457,29.938827,-6.104167,-0.354167,-8.0,-2.5,12.711596,9.244871,-5.900000,0.071429,-9.0,-3.0,15.036333,11.571230,1.238095,5.888889,-8.0,-2.0,26.633641,22.989118,2.150943,6.943396,-7.0,-2.0,30.645995,28.491212
3,2018-05-29,DL,DL,DL,678,DL,N365NB,678,11996,GSP,"Greer, SC",11433,DTW,"Detroit, MI",0600,-5.0,0750,N,110.0,1,5

# Columns dropping starts here
## Convert categorical data

### Get dummies

In [ ]:
#Get dummies
df_dummy = pd.get_dummies(dfF[["op_unique_carrier"]])
df_dummy.shape
# Merge into DF
dfF = pd.concat([dfF,df_dummy],axis=1).drop(["op_unique_carrier",'op_unique_carrier_9E'],axis=1)

#Get dummies
df_dummy = pd.get_dummies(dfFT[["op_unique_carrier"]])
df_dummy.shape
# Merge into DF
dfFT = pd.concat([dfFT,df_dummy],axis=1).drop(["op_unique_carrier",'op_unique_carrier_9E'],axis=1)

### Drop columns

In [ ]:
for c in dfF.columns:
    print(c)

fl_date
mkt_unique_carrier
branded_code_share
mkt_carrier
mkt_carrier_fl_num
tail_num
op_carrier_fl_num
origin_airport_id
origin
origin_city_name
dest_airport_id
dest
dest_city_name
crs_dep_time
dep_delay
crs_arr_time
dup
crs_elapsed_time
flights
distance
arr_delay
carrier_delay
weather_delay
nas_delay
security_delay
late_aircraft_delay
cancelled
iscarrier_delay
isweather_delay
isnas_delay
issecurity_delay
islate_aircraft_delay
flight_number
fl_month
fl_day
fl_day_of_year
fl_day_of_week
dep_hour
arr_hour
duration_type
dep_daysection
arr_daysection
fl_season
origin_state
dest_state
origin_airport_vol
dest_airport_vol
carrier_origin_apt_vol
carrier_dest_apt_vol
carrier_origin_apt_pct
carrier_dest_apt_pct
mean_carrier_arr_delay
mean_carrier_arr_delay_month
mean_carrier_dep_delay_month
median_carrier_arr_delay_month
median_carrier_dep_delay_month
std_carrier_arr_delay_month
std_carrier_dep_delay_month
mean_carrier_origin_dep-hour_dep_delay
mean_carrier_dest_fl-hour_delay
median_carrier_ori

In [ ]:
for c in dfFT.columns:
    print(c)

fl_date
mkt_unique_carrier
branded_code_share
mkt_carrier
mkt_carrier_fl_num
tail_num
op_carrier_fl_num
origin_airport_id
origin
origin_city_name
dest_airport_id
dest
dest_city_name
crs_dep_time
crs_arr_time
dup
crs_elapsed_time
flights
distance
flight_number
fl_month
fl_day
fl_day_of_year
fl_day_of_week
dep_hour
arr_hour
duration_type
dep_daysection
arr_daysection
fl_season
origin_state
dest_state
origin_airport_vol
dest_airport_vol
carrier_origin_apt_vol
carrier_dest_apt_vol
carrier_origin_apt_pct
carrier_dest_apt_pct
mean_carrier_arr_delay
mean_carrier_arr_delay_month
mean_carrier_dep_delay_month
median_carrier_arr_delay_month
median_carrier_dep_delay_month
std_carrier_arr_delay_month
std_carrier_dep_delay_month
mean_carrier_origin_dep-hour_dep_delay
mean_carrier_dest_fl-hour_delay
median_carrier_origin_dep-hour_dep_delay
median_carrier_dest_fl-hour_delay
std_carrier_origin_dep-hour_dep_delay
std_carrier_dest_fl-hour_delay
mean_carrier_route_month_dow_dayS_dep-delay
mean_carrier_rou

In [ ]:
dfF1 = dfF[[
    'fl_date',
    'mkt_unique_carrier',
    'branded_code_share',
    'mkt_carrier',
    'mkt_carrier_fl_num',
    'tail_num',
    'op_carrier_fl_num',
    'origin_airport_id',
    'origin',
    'origin_city_name',
    'dest_airport_id',
    'dest',
    'dest_city_name',
    'crs_dep_time',
    'dep_delay',
    'crs_arr_time',
    'dup',
    'crs_elapsed_time',
    'flights',
    'distance',
    'arr_delay',
    'carrier_delay',
    'weather_delay',
    'nas_delay',
    'security_delay',
    'late_aircraft_delay',
    'cancelled',
    'iscarrier_delay',
    'isweather_delay',
    'isnas_delay',
    'issecurity_delay',
    'islate_aircraft_delay',
    'flight_number',
    'fl_month',
    'fl_day',
    'fl_day_of_year',
    'fl_day_of_week',
    'dep_hour',
    'arr_hour',
    'duration_type',
    'dep_daysection',
    'arr_daysection',
    'fl_season',
    'origin_state',
    'dest_state',
    'origin_airport_vol',
    'dest_airport_vol',
    'carrier_origin_apt_vol',
    'carrier_dest_apt_vol',
    'carrier_origin_apt_pct',
    'carrier_dest_apt_pct',
    'mean_carrier_arr_delay',
    'mean_carrier_arr_delay_month',
    'mean_carrier_dep_delay_month',
    'median_carrier_arr_delay_month',
    'median_carrier_dep_delay_month',
    'std_carrier_arr_delay_month',
    'std_carrier_dep_delay_month',
    'mean_carrier_origin_dep-hour_dep_delay',
    'mean_carrier_dest_fl-hour_delay',
    'median_carrier_origin_dep-hour_dep_delay',
    'median_carrier_dest_fl-hour_delay',
    'std_carrier_origin_dep-hour_dep_delay',
    'std_carrier_dest_fl-hour_delay',
    'mean_carrier_route_month_dow_dayS_dep-delay',
    'mean_carrier_route_month_dow_dayS_arr-delay',
    'median_carrier_route_month_dow_dayS_dep-delay',
    'median_carrier_route_month_dow_dayS_arr-delay',
    'std_carrier_route_month_dow_dayS_dep-delay',
    'std_carrier_route_month_dow_dayS_arr-delay',
    'mean_route_arr_delay_month',
    'mean_route_dep_delay_month',
    'mean_route_arr_delay_dow',
    'mean_route_dep_delay_dow',
    'mean_route_arr_delay_dep_hour',
    'mean_route_dep_delay_dep_hour',
    'mean_route_arr_delay_arr_hour',
    'mean_route_dep_delay_arr_hour',
    'median_route_arr_delay_month',
    'median_route_dep_delay_month',
    'median_route_arr_delay_dow',
    'median_route_dep_delay_dow',
    'median_route_arr_delay_dep_hour',
    'median_route_dep_delay_dep_hour',
    'median_route_arr_delay_arr_hour',
    'median_route_dep_delay_arr_hour',
    'std_route_arr_delay_month',
    'std_route_dep_delay_month',
    'std_route_arr_delay_dow',
    'std_route_dep_delay_dow',
    'std_route_arr_delay_dep_hour',
    'std_route_dep_delay_dep_hour',
    'std_route_arr_delay_arr_hour',
    'std_route_dep_delay_arr_hour',
    'op_unique_carrier_9K',
    'op_unique_carrier_AA',
    'op_unique_carrier_AS',
    'op_unique_carrier_AX',
    'op_unique_carrier_B6',
    'op_unique_carrier_C5',
    'op_unique_carrier_CP',
    'op_unique_carrier_DL',
    'op_unique_carrier_EM',
    'op_unique_carrier_EV',
    'op_unique_carrier_F9',
    'op_unique_carrier_G4',
    'op_unique_carrier_G7',
    'op_unique_carrier_HA',
    'op_unique_carrier_KS',
    'op_unique_carrier_MQ',
    'op_unique_carrier_NK',
    'op_unique_carrier_OH',
    'op_unique_carrier_OO',
    'op_unique_carrier_PT',
    'op_unique_carrier_QX',
    'op_unique_carrier_UA',
    'op_unique_carrier_VX',
    'op_unique_carrier_WN',
    'op_unique_carrier_YV',
    'op_unique_carrier_YX',
    'op_unique_carrier_ZW'
]]

#### Select numerical colums, drop columns with Nans

In [ ]:
dfF1 = dfF1.select_dtypes('number').dropna(axis='columns')

In [ ]:
dfFT1 = dfFT[[
        'mean_carrier_origin_dep-hour_dep_delay',
       'mean_carrier_dest_fl-hour_delay', 'median_carrier_dest_fl-hour_delay',
       'mean_carrier_route_month_dow_dayS_dep-delay',
       'mean_carrier_route_month_dow_dayS_arr-delay',
       'mean_route_arr_delay_month', 'mean_route_arr_delay_dow',
       'mean_route_arr_delay_dep_hour', 'mean_route_arr_delay_arr_hour',
       'median_route_arr_delay_month', 'median_route_dep_delay_month',
       'median_route_arr_delay_dow', 'median_route_arr_delay_dep_hour',
       'median_route_arr_delay_arr_hour']]

In [ ]:
#dfF1

## Deal with Naans

In [ ]:
total = dfF1.isnull().sum().sort_values(ascending=False)
percent = (dfF1.isnull().sum()/dfF1.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(50)

,Total,Percent
op_unique_carrier_ZW,0,0.0
median_carrier_arr_delay_month,0,0.0
dest_airport_vol,0,0.0
carrier_origin_apt_vol,0,0.0
carrier_dest_apt_vol,0,0.0
carrier_origin_apt_pct,0,0.0
carrier_dest_apt_pct,0,0.0
mean_carrier_arr_delay,0,0.0
mean_carrier_arr_delay_month,0,0.0
mean_carrier_dep_delay_month,0,0.0


In [ ]:
#dfF1 = dfF1[dfF1['arr_delay'].notnull()]
#dfF1 = dfF1[dfF1['crs_elapsed_time'].notnull()]

In [ ]:
total = dfFT1.isnull().sum().sort_values(ascending=False)
percent = (dfFT1.isnull().sum()/dfFT1.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
median_route_arr_delay_arr_hour,13524,0.020474
median_route_arr_delay_dep_hour,13524,0.020474
median_route_arr_delay_dow,13524,0.020474
median_route_dep_delay_month,13524,0.020474
median_route_arr_delay_month,13524,0.020474
mean_route_arr_delay_arr_hour,13524,0.020474
mean_route_arr_delay_dep_hour,13524,0.020474
mean_route_arr_delay_dow,13524,0.020474
mean_route_arr_delay_month,13524,0.020474
mean_carrier_route_month_dow_dayS_arr-delay,13524,0.020474


### removing 2% Nans - Lets come back here if we have time. 

In [ ]:
dfFT1 =dfFT1.apply(lambda x: x.fillna(x.median()),axis=0)

In [ ]:
total = dfFT1.isnull().sum().sort_values(ascending=False)
percent = (dfFT1.isnull().sum()/dfFT1.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
median_route_arr_delay_arr_hour,0,0.0
median_route_arr_delay_dep_hour,0,0.0
median_route_arr_delay_dow,0,0.0
median_route_dep_delay_month,0,0.0
median_route_arr_delay_month,0,0.0
mean_route_arr_delay_arr_hour,0,0.0
mean_route_arr_delay_dep_hour,0,0.0
mean_route_arr_delay_dow,0,0.0
mean_route_arr_delay_month,0,0.0
mean_carrier_route_month_dow_dayS_arr-delay,0,0.0


### Proceed with Variance Threshold, Highly correlated pairs removal, and SelectKBest

In [ ]:
y = dfF1.arr_delay
df_numeric = dfF1.drop(["arr_delay"],axis=1)

from sklearn.feature_selection import VarianceThreshold
# remove features with small variance
vt = VarianceThreshold(0.1)
df_transformed = vt.fit_transform(df_numeric)
selected_columns = df_numeric.columns[vt.get_support()]
# transforming an array back to a data-frame preserves column labels
df_transformed = pd.DataFrame(df_transformed, columns = selected_columns)

In [ ]:
# remove highly correlated pairs
df_corr = df_transformed.corr().abs()
indices = np.where(df_corr > 0.9)
indices = [(df_corr.index[x], df_corr.columns[y]) for x, y in zip(*indices)
              if x != y and x < y]
for idx in indices:
    try:
        df_transformed.drop(idx[1], axis = 1, inplace=True)
    except KeyError:
        pass

In [ ]:
# forward regression
from sklearn.feature_selection import f_regression, SelectKBest
skb = SelectKBest(f_regression, k=15)
X = skb.fit_transform(df_transformed, y)
X = pd.DataFrame(X,columns=df_transformed.columns[skb.get_support()])
X

,dep_delay,mean_carrier_origin_dep-hour_dep_delay,mean_carrier_dest_fl-hour_delay,median_carrier_dest_fl-hour_delay,mean_carrier_route_month_dow_dayS_dep-delay,mean_carrier_route_month_dow_dayS_arr-delay,mean_route_arr_delay_month,mean_route_arr_delay_dow,mean_route_arr_delay_dep_hour,mean_route_arr_delay_arr_hour,median_route_arr_delay_month,median_route_dep_delay_month,median_route_arr_delay_dow,median_route_arr_delay_dep_hour,median_route_arr_delay_arr_hour
0,31.0,16.668478,7.911765,1.5,17.000000,13.000000,0.444444,-4.957746,7.257353,7.911765,-4.0,-2.0,-10.0,-5.0,1.5
1,-6.0,3.293103,-7.638963,-11.0,-9.666667,-15.666667,-0.625000,-2.330275,-5.790875,-8.484472,-6.5,-4.0,-9.0,-11.0,-11.0
2,26.0,13.427613,9.351351,-3.0,8.000000,8.666667,-6.104167,-5.900000,1.238095,2.150943,-8.0,-2.5,-9.0,-8.0,-7.0
3,-5.0,9.800000,-7.638963,-11.0,-5.000000,-21.000000,-10.411765,-5.000000,-13.921875,-12.911765,-15.0,-6.0,-11.0,-15.0,-14.0
4,41.0,7.101957,-0.241935,-4.0,20.500000,20.000000,-5.944444,1.272727,-1.118421,-1.118421,-12.0,-3.0,-10.0,-4.0,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560687,-3.0,8.630597,2.689552,-5.0,-1.250000,9.333333,0.252294,-6.025063,-4.276551,-4.620397,-11.0,-3.0,-14.0,-12.0,-14.0
1560688,-3.0,22.066308,7.109890,-2.0,-4.000000,5.666667,0.252294,-6.025063,-2.402204,1.437819,-11.0,-3.0,-14.0,-13.0,-11.0
1560689,-6.0,10.362924,-1.751479,-10.0,-6.000000,-32.000000,-1.833333,-4.363636,-2.242424,-0.816667,-8.5,0.0,-18.0,-10.0,-9.5
1560690,16.0,5.492141,-3.379791,-13.0,8.000000,0.666667,10.883333,3.229167,7.628319,1.595420,-1.0,0.0,-7.0,-7.0,-7.0


#### Remove the forbidden dep_delay column

In [ ]:
X = X.drop('dep_delay',axis=1)

In [ ]:
X.columns

Index(['mean_carrier_origin_dep-hour_dep_delay',
       'mean_carrier_dest_fl-hour_delay', 'median_carrier_dest_fl-hour_delay',
       'mean_carrier_route_month_dow_dayS_dep-delay',
       'mean_carrier_route_month_dow_dayS_arr-delay',
       'mean_route_arr_delay_month', 'mean_route_arr_delay_dow',
       'mean_route_arr_delay_dep_hour', 'mean_route_arr_delay_arr_hour',
       'median_route_arr_delay_month', 'median_route_dep_delay_month',
       'median_route_arr_delay_dow', 'median_route_arr_delay_dep_hour',
       'median_route_arr_delay_arr_hour'],
      dtype='object')

# Modeling Question 1

In [ ]:
# # Assign X and y
# X = dfF1.drop('arr_delay',axis=1)
# y = dfF1['arr_delay']

In [ ]:
# Data test train Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.80,random_state=1)


## RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=40, max_depth=6, random_state=0)
regr.fit(X_train,y_train)

RandomForestRegressor(max_depth=6, n_estimators=40, random_state=0)

In [ ]:
y_pred=regr.predict(X_test)

In [ ]:
#Mean Absolute Error
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test, y_pred))

#Mean Squared Error
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, y_pred))

#R2 Score
from sklearn.metrics import r2_score
print(r2_score(y_test, y_pred))

10.34954239864604
675.0603117407289
0.7240114794517591


In [ ]:
pd.Series(y_pred).describe()

count    312139.000000
mean          5.428579
std          41.767766
min         -43.824506
25%         -12.992054
50%          -4.287721
75%           9.532076
max        1464.019054
dtype: float64

In [ ]:
# Try grid search if we have time. 


# Final flights_test.csv prediction

In [ ]:
final_pred=regr.predict(dfFT1)

In [ ]:
predicted_delay = pd.DataFrame(final_pred).rename(columns={0: 'predicted_delay'})
predicted_delay

,predicted_delay
0,12.524110
1,-15.626683
2,6.765955
3,-21.752039
4,22.044413
...,...
660551,-8.408843
660552,-18.332913
660553,-10.608048
660554,-10.657608


In [ ]:
submission = pd.read_csv(str(path)+"flights_test.csv",low_memory=False)

In [ ]:
submission = submission[['fl_date','mkt_carrier','mkt_carrier_fl_num','origin','dest']]

In [ ]:
submission = pd.concat([submission, predicted_delay], axis=1)

In [ ]:
submission.to_csv('../../../data/Q1submission.csv')

In [ ]:
# Dealing with Ram

In [ ]:

import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

#say a data frame is taking up a lot and it happens to be one you don't need anymore you can do:
#del df

[('dfF', 2602606480),
 ('df', 1542050435),
 ('dfFT', 1007882019),
 ('dfF1', 820728364),
 ('df_numeric', 808242828),
 ('df_transformed', 524392656),
 ('df_test', 507999370),
 ('dfFT1', 197506388),
 ('X', 174797648),
 ('X_train', 149826376),
 ('y', 66914128),
 ('X_test', 37456696),
 ('y_train', 19976864),
 ('df_dummy', 16514044),
 ('y_test', 4994240),
 ('final_pred', 2497208),
 ('y_pred', 2497208),
 ('df_corr', 26808),
 ('selected_columns', 6896),
 ('RandomForestRegressor', 2008),
 ('missing_data', 1805),
 ('percent', 1661),
 ('total', 1661),
 ('hour_to_daysection', 1176),
 ('SelectKBest', 1064),
 ('VarianceThreshold', 1064),
 ('month_to_season', 640),
 ('indices', 184),
 ('f_regression', 136),
 ('mean_absolute_error', 136),
 ('mean_squared_error', 136),
 ('r2_score', 136),
 ('train_test_split', 136),
 ('delaytypes', 96),
 ('bins', 88),
 ('labels', 80),
 ('np', 72),
 ('pd', 72),
 ('preprocessing', 72),
 ('c', 69),
 ('delay', 68),
 ('path', 63),
 ('idx', 56),
 ('regr', 48),
 ('skb', 48),


In [ ]:
del dfF_encodertest
del dfF_encoded
del df_test
del dfF1

NameError: name 'dfF_encodertest' is not defined